In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def MSE(y, tx, w):
    """Compute MSE at w
    
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,D+1)
        w: numpy array of shape=(D+1, ). The vector of model parameters.
        
    Returns:
        Returns the mean square error at w for input tx and output y
    """
    e = y - tx.dot(w)
    return np.mean(e**2)

In [5]:
def compute_gradient(y, tx, w):
    """Computes the gradient at w.
        
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,D+1)
        w: numpy array of shape=(D+1, ). The vector of model parameters.
        
    Returns:
        An numpy array of shape (D+1, ) (same shape as w), containing the gradient of the loss at w.
    """
    e = y - tx.dot(w)
    return -tx.T.dot(e)/len(y)

In [6]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """The Gradient Descent (GD) algorithm for least squares.
        
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,D+1)
        initial_w: numpy array of shape=(D+1, ). The initial guess (or the initialization) for the model parameters
        max_iters: a scalar denoting the total number of iterations of GD
        gamma: a scalar denoting the stepsize
        
    Returns:
        w: the model parameter as numpy arrays of shape (2, ), for the last iteration of GD 
        loss: the loss value corresponding to w
    """
    
    w = initial_w
    for n_iter in range(max_iters):
        grad = compute_gradient(y, tx, w)
        loss = MSE(y, tx, w)
        w = w - gamma * grad

    return w, loss

In [ ]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):